In [1]:
%reset -f
import pandas as pd 
import csv
import numpy as np

In [2]:
#https://stackoverflow.com/questions/50165501/generate-random-list-of-timestamps-in-python
import random
def randomTime():
    # generate random number scaled to number of seconds in a day
    # (24*60*60) = 86,400

    rtime = int(random.random()*86400)

    hours   = int(rtime/3600)
    minutes = int((rtime - hours*3600)/60)
    seconds = rtime - hours*3600 - minutes*60

    time_string = '%02d:%02d:%02d' % (hours, minutes, seconds)
    return time_string

In [3]:
df = pd.DataFrame({'UserID':['a','a','b','b','a','a','a'],
     'Time':[1,2,1,2,1,2,3],
     'data1':np.random.randn(7),
     'data2':np.random.randn(7),
     })

df.head(10)

,UserID,Time,data1,data2
0,a,1,0.194980,-0.606535
1,a,2,-0.090859,-0.401919
2,b,1,2.560549,1.921460
3,b,2,-0.969063,0.658547
4,a,1,-0.616020,-0.882137
5,a,2,-1.765363,-1.186519
6,a,3,1.079963,-0.322701


In [4]:
df['DateTime'] = df['UserID'].transform(lambda x : randomTime())
df['DateTime'] = pd.to_datetime(df['DateTime'])
df = df.sort_values(['UserID','DateTime','Time'])

df.head(10)

,UserID,Time,data1,data2,DateTime
1,a,2,-0.090859,-0.401919,2020-06-17 01:07:20
4,a,1,-0.616020,-0.882137,2020-06-17 10:40:18
5,a,2,-1.765363,-1.186519,2020-06-17 12:17:51
0,a,1,0.194980,-0.606535,2020-06-17 18:36:53
6,a,3,1.079963,-0.322701,2020-06-17 23:08:01
3,b,2,-0.969063,0.658547,2020-06-17 04:19:30
2,b,1,2.560549,1.921460,2020-06-17 11:54:17


In [5]:
#GroupingTime
df['DateTime_pre1'] = df.groupby(['UserID'])['DateTime'].transform(lambda x :x.shift(1))
df['DateTime_dif'] = df['DateTime'] - df['DateTime_pre1']
df['DateTime_dif_delta'] = df['DateTime_dif'].astype('timedelta64[m]')

df.head(10)

,UserID,Time,data1,data2,DateTime,DateTime_pre1,DateTime_dif,DateTime_dif_delta
1,a,2,-0.090859,-0.401919,2020-06-17 01:07:20,NaT,NaT,NaN
4,a,1,-0.616020,-0.882137,2020-06-17 10:40:18,2020-06-17 01:07:20,09:32:58,572.0
5,a,2,-1.765363,-1.186519,2020-06-17 12:17:51,2020-06-17 10:40:18,01:37:33,97.0
0,a,1,0.194980,-0.606535,2020-06-17 18:36:53,2020-06-17 12:17:51,06:19:02,379.0
6,a,3,1.079963,-0.322701,2020-06-17 23:08:01,2020-06-17 18:36:53,04:31:08,271.0
3,b,2,-0.969063,0.658547,2020-06-17 04:19:30,NaT,NaT,NaN
2,b,1,2.560549,1.921460,2020-06-17 11:54:17,2020-06-17 04:19:30,07:34:47,454.0


In [6]:
#看第t筆UserID是否跟t+1筆相同
df['UserID_lag1'] = df['UserID'].shift(1)
df['Time_lag1'] = df.groupby(['UserID'])['Time'].shift(1)

df.head(10)

,UserID,Time,data1,data2,DateTime,DateTime_pre1,DateTime_dif,DateTime_dif_delta,UserID_lag1,Time_lag1
1,a,2,-0.090859,-0.401919,2020-06-17 01:07:20,NaT,NaT,NaN,NaN,NaN
4,a,1,-0.616020,-0.882137,2020-06-17 10:40:18,2020-06-17 01:07:20,09:32:58,572.0,a,2.0
5,a,2,-1.765363,-1.186519,2020-06-17 12:17:51,2020-06-17 10:40:18,01:37:33,97.0,a,1.0
0,a,1,0.194980,-0.606535,2020-06-17 18:36:53,2020-06-17 12:17:51,06:19:02,379.0,a,2.0
6,a,3,1.079963,-0.322701,2020-06-17 23:08:01,2020-06-17 18:36:53,04:31:08,271.0,a,1.0
3,b,2,-0.969063,0.658547,2020-06-17 04:19:30,NaT,NaT,NaN,a,NaN
2,b,1,2.560549,1.921460,2020-06-17 11:54:17,2020-06-17 04:19:30,07:34:47,454.0,b,2.0


In [7]:
#transform method:一次處理一欄 
df['ID_data1_mean'] = df.groupby(['UserID'])['data1'].transform(np.mean)

df.head(10)

,UserID,Time,data1,data2,DateTime,DateTime_pre1,DateTime_dif,DateTime_dif_delta,UserID_lag1,Time_lag1,ID_data1_mean
1,a,2,-0.090859,-0.401919,2020-06-17 01:07:20,NaT,NaT,NaN,NaN,NaN,-0.239460
4,a,1,-0.616020,-0.882137,2020-06-17 10:40:18,2020-06-17 01:07:20,09:32:58,572.0,a,2.0,-0.239460
5,a,2,-1.765363,-1.186519,2020-06-17 12:17:51,2020-06-17 10:40:18,01:37:33,97.0,a,1.0,-0.239460
0,a,1,0.194980,-0.606535,2020-06-17 18:36:53,2020-06-17 12:17:51,06:19:02,379.0,a,2.0,-0.239460
6,a,3,1.079963,-0.322701,2020-06-17 23:08:01,2020-06-17 18:36:53,04:31:08,271.0,a,1.0,-0.239460
3,b,2,-0.969063,0.658547,2020-06-17 04:19:30,NaT,NaT,NaN,a,NaN,0.795743
2,b,1,2.560549,1.921460,2020-06-17 11:54:17,2020-06-17 04:19:30,07:34:47,454.0,b,2.0,0.795743


In [8]:
#兩欄的差
df['data_dif'] = df['data1']-df['data2']
df['rolling_mean'] = df.groupby(['UserID'])['data_dif'].transform(lambda x : x.rolling(2).mean())

df.head(10)

,UserID,Time,data1,data2,DateTime,DateTime_pre1,DateTime_dif,DateTime_dif_delta,UserID_lag1,Time_lag1,ID_data1_mean,data_dif,rolling_mean
1,a,2,-0.090859,-0.401919,2020-06-17 01:07:20,NaT,NaT,NaN,NaN,NaN,-0.239460,0.311059,NaN
4,a,1,-0.616020,-0.882137,2020-06-17 10:40:18,2020-06-17 01:07:20,09:32:58,572.0,a,2.0,-0.239460,0.266117,0.288588
5,a,2,-1.765363,-1.186519,2020-06-17 12:17:51,2020-06-17 10:40:18,01:37:33,97.0,a,1.0,-0.239460,-0.578844,-0.156363
0,a,1,0.194980,-0.606535,2020-06-17 18:36:53,2020-06-17 12:17:51,06:19:02,379.0,a,2.0,-0.239460,0.801515,0.111335
6,a,3,1.079963,-0.322701,2020-06-17 23:08:01,2020-06-17 18:36:53,04:31:08,271.0,a,1.0,-0.239460,1.402665,1.102090
3,b,2,-0.969063,0.658547,2020-06-17 04:19:30,NaT,NaT,NaN,a,NaN,0.795743,-1.627610,NaN
2,b,1,2.560549,1.921460,2020-06-17 11:54:17,2020-06-17 04:19:30,07:34:47,454.0,b,2.0,0.795743,0.639089,-0.494261


In [9]:
#Grouping mean
df.groupby('UserID').mean().add_prefix('mean_')

df.head(10)

,UserID,Time,data1,data2,DateTime,DateTime_pre1,DateTime_dif,DateTime_dif_delta,UserID_lag1,Time_lag1,ID_data1_mean,data_dif,rolling_mean
1,a,2,-0.090859,-0.401919,2020-06-17 01:07:20,NaT,NaT,NaN,NaN,NaN,-0.239460,0.311059,NaN
4,a,1,-0.616020,-0.882137,2020-06-17 10:40:18,2020-06-17 01:07:20,09:32:58,572.0,a,2.0,-0.239460,0.266117,0.288588
5,a,2,-1.765363,-1.186519,2020-06-17 12:17:51,2020-06-17 10:40:18,01:37:33,97.0,a,1.0,-0.239460,-0.578844,-0.156363
0,a,1,0.194980,-0.606535,2020-06-17 18:36:53,2020-06-17 12:17:51,06:19:02,379.0,a,2.0,-0.239460,0.801515,0.111335
6,a,3,1.079963,-0.322701,2020-06-17 23:08:01,2020-06-17 18:36:53,04:31:08,271.0,a,1.0,-0.239460,1.402665,1.102090
3,b,2,-0.969063,0.658547,2020-06-17 04:19:30,NaT,NaT,NaN,a,NaN,0.795743,-1.627610,NaN
2,b,1,2.560549,1.921460,2020-06-17 11:54:17,2020-06-17 04:19:30,07:34:47,454.0,b,2.0,0.795743,0.639089,-0.494261


In [10]:
#example of ''.join()
str1 = "-";seq = ("a", "b", "c") 
print (str1.join( seq ))

a-b-c


In [12]:
#lambda UserID.unique
#GroupbyDataFrame['UserID']
#'UserID':[index] -> 'UserID':[0 4 1 5 6] 
df['Time_str'] = df['Time'].astype(str)
df_time = df.groupby(['UserID'])['Time_str'].apply(lambda x: x.unique())
df_time_join = df_time.apply(lambda x: '&'.join(x))

df_time_join.head(10)

UserID
a    2&1&3
b      2&1
Name: Time_str, dtype: object